# Spectrum Licence Composite Dataset Construction

The Register of Radiocommunication Licences (RRL) is database containing licensing information spread across various .csv files. The information stored in the various csv files need to be merged into a single dataset using a licence number index.

### Desire Spectrum Licence Information
- Licence number
- Client number
- Frequency Ranges
- Bandwidth
- HCIS Cells

In [21]:
import pandas as pd
import numpy as np

# getRRL() -> To be implemented. Downloads RRL and verifies structure is compatible with specNav.

# -----------------
# def getSpecLic ():
# -----------------

# From licence database, create dataset for spectrum licences
licence = pd.read_csv('licence.csv')


# Column tidy
licence.drop(columns=[
    'SV_ID',
    'SS_ID',
    'DATE_ISSUED',
    'DATE_OF_EFFECT',
    'DATE_OF_EXPIRY',
    'STATUS',
    'STATUS_TEXT',
    'AP_ID',
    'AP_PRJ_IDENT',
    'SHIP_NAME',
    'BSL_NO'
    ], inplace=True)

# Isolate and sort Spectrum Licence Entries
specLicence = licence.loc[licence['LICENCE_TYPE_NAME'] == 'Spectrum']
specLicence.reset_index(drop=True, inplace=True)
specLicence = specLicence.astype({'LICENCE_NO':'int64'})
specLicence
    

,LICENCE_NO,CLIENT_NO,LICENCE_TYPE_NAME,LICENCE_CATEGORY_NAME
0,10143110,1136980.0,Spectrum,2 GHz Band
1,10143136,1103274.0,Spectrum,2 GHz Band
2,10143466,1104504.0,Spectrum,2 GHz Band
3,10143562,1103276.0,Spectrum,2 GHz Band
4,10231258,1103276.0,Spectrum,1800 MHz Band
...,...,...,...,...
161,9469870,1149289.0,Spectrum,2.5 GHz Band
162,9469871,1103275.0,Spectrum,2.5 GHz Band
163,9469878,1103275.0,Spectrum,2.5 GHz Band
164,9599659,1103275.0,Spectrum,2.3 GHz Band


In [22]:
# Frequency dataset
freq = pd.read_csv('auth_spectrum_freq.csv')
freq.drop(columns=['AREA_NAME'], inplace=True)
freq = freq.astype({'LICENCE_NO':'int64'})
freq

,LICENCE_NO,AREA_CODE,LW_FREQUENCY_START,LW_FREQUENCY_END,UP_FREQUENCY_START,UP_FREQUENCY_END
0,10006309,AP_10006309_3714,1710000000,1712500000,1.805000e+09,1.807500e+09
1,10143110,AP_10143110_3918,1960000000,1970000000,2.150000e+09,2.160000e+09
2,10143110,AP_10143110_3919,1960000000,1970000000,2.150000e+09,2.160000e+09
3,10143110,AP_10143110_3920,1960000000,1970000000,2.150000e+09,2.160000e+09
4,10143110,AP_10143110_3916,1920000000,1935000000,2.110000e+09,2.125000e+09
...,...,...,...,...,...,...
1468,9599659,AP_9599659_2778,2302000000,2337000000,NaN,NaN
1469,9619844,AP_9619844_2798,1740000000,1755000000,1.835000e+09,1.850000e+09
1470,9619844,AP_9619844_2800,1745000000,1755000000,1.840000e+09,1.850000e+09
1471,9619844,AP_9619844_2799,1740000000,1755000000,1.835000e+09,1.850000e+09


In [26]:
# Merge frequency dataset on spectrum licence dataset index
SpecFreq = freq[freq['LICENCE_NO'].isin(specLicence['LICENCE_NO'])]
bandNames = specLicence[['LICENCE_NO','LICENCE_CATEGORY_NAME']]
#SpecFreq = pd.merge(SpecFreq, bandNames, how='left', on='LICENCE_NO')
SpecFreq

,LICENCE_NO,AREA_CODE,LW_FREQUENCY_START,LW_FREQUENCY_END,UP_FREQUENCY_START,UP_FREQUENCY_END
1,10143110,AP_10143110_3918,1960000000,1970000000,2.150000e+09,2.160000e+09
2,10143110,AP_10143110_3919,1960000000,1970000000,2.150000e+09,2.160000e+09
3,10143110,AP_10143110_3920,1960000000,1970000000,2.150000e+09,2.160000e+09
4,10143110,AP_10143110_3916,1920000000,1935000000,2.110000e+09,2.125000e+09
5,10143110,AP_10143110_3917,1920000000,1935000000,2.110000e+09,2.125000e+09
...,...,...,...,...,...,...
1467,9599659,AP_9599659_2777,2365000000,2400000000,NaN,NaN
1468,9599659,AP_9599659_2778,2302000000,2337000000,NaN,NaN
1469,9619844,AP_9619844_2798,1740000000,1755000000,1.835000e+09,1.850000e+09
1470,9619844,AP_9619844_2800,1745000000,1755000000,1.840000e+09,1.850000e+09


# Spectrum Licence Dataset Querying
Using the composite dataset, generate a sorted dataset containing only the licences for a given list of client numbers. For improved accessibility, sort licences by client number (if multiple client numbers), HCIS ID's and frequency ranges. Implement grouping of entries if possible.